In [4]:
!pip install ipywidgets


Defaulting to user installation because normal site-packages is not writeable


In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import random

# Load and preprocess dataset
def load_and_preprocess_data():
    try:
        # Load datasets
        df_bert = pd.read_csv('C:/Users/suru8/OneDrive/Desktop/Project/u2361989/Bert.csv')[['condition', 'urlDrugName', 'effectiveness']]
        df_drugs_recommendation = pd.read_csv('C:/Users/suru8/OneDrive/Desktop/Project/u2361989/drugs_recommendation.csv')[['condition', 'urlDrugName', 'effectiveness']]
        df_top_2 = pd.read_csv('C:/Users/suru8/OneDrive/Desktop/Project/u2361989/top_2_drugs_recommendation.csv')[['condition', 'urlDrugName', 'composite_score']]

        # Rename 'composite_score' to 'effectiveness' for consistency
        df_top_2 = df_top_2.rename(columns={'composite_score': 'effectiveness'})

        # Combine the datasets
        combined_df = pd.concat([df_bert, df_drugs_recommendation, df_top_2], ignore_index=True)

        # Clean the data: convert effectiveness to numeric and drop rows with NaN values
        combined_df['effectiveness'] = pd.to_numeric(combined_df['effectiveness'], errors='coerce')
        combined_df = combined_df.dropna(subset=['effectiveness'])

        return combined_df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

# Function to get top 3 unique drug recommendations based on condition with reasoning
def get_drug_recommendation(condition):
    # Load and preprocess the dataset before every recommendation
    df = load_and_preprocess_data()
    if df is None or df.empty:
        return "Sorry, the drug data is unavailable at the moment."

    condition = condition.lower()
    
    # Filter the dataset by condition and ensure only unique drugs
    results = df[df['condition'].str.contains(condition, case=False, na=False)]
    results = results.drop_duplicates(subset='urlDrugName')  # Ensure unique drugs
    results = results.nlargest(3, 'effectiveness')  # Get top 3 drugs by effectiveness
    
    if results.empty:
        return f"Sorry, I don't have drug recommendations for '{condition}'."
    
    # Build response with HTML list format
    response = f"<b>Top 3 drugs for {condition.capitalize()}:</b><ul>"
    
    # Explain why the top drug is recommended
    top_drug = results.iloc[0]
    reasoning = get_reasoning_for_recommendation(top_drug)
    
    # Create the list of top 3 drugs with new numbering (starting from 1)
    for idx, row in enumerate(results.itertuples(), 1):
        if idx == 1:
            response += f"<li>{row.urlDrugName} with effectiveness: {row.effectiveness:.2f} (Recommended: {reasoning})</li>"
        else:
            response += f"<li>{row.urlDrugName} with effectiveness: {row.effectiveness:.2f}</li>"
    
    response += "</ul>"  # Close the unordered list
    
    return response

# Function to provide reasoning for recommending the top drug
def get_reasoning_for_recommendation(drug_row):
    # Placeholder logic for reasoning - you can expand this based on more data like side effects, cost, etc.
    
    reasons = [
        f"{drug_row.urlDrugName} has the highest effectiveness score.",
        f"{drug_row.urlDrugName} is commonly prescribed for {drug_row.condition.capitalize()}.",
        f"{drug_row.urlDrugName} is one of the most frequently recommended drugs in multiple datasets.",
        f"{drug_row.urlDrugName} has shown positive results in clinical trials for {drug_row.condition.capitalize()}."
    ]
    
    # Randomly select a reasoning from the list for demonstration purposes
    return random.choice(reasons)

# Initialize chatbox to store chat messages
chatbox = widgets.VBox([])

# Function to handle chatbot interaction and simulate a pop-up effect
def handle_chatbot_interaction(condition):
    if condition.lower() in ['exit', 'quit', 'bye']:
        chatbox.children += (widgets.HTML(f"<b>Assistant:</b> Goodbye!"),)
        return False  # Signal to stop the chatbot
    
    if condition.lower() == 'help':
        help_message = "Enter a medical condition to get top 3 drug recommendations, or type 'exit' to quit."
        chatbox.children += (widgets.HTML(f"<b>Assistant:</b> {help_message}"),)
        return True  # Continue the chatbot
    
    # Get drug recommendation
    response = get_drug_recommendation(condition)
    
    # Add the user message and the assistant response to the chat history
    chatbox.children += (
        widgets.HTML(f"<b>User:</b> {condition}"),  # User input
        widgets.HTML(f"<b>Assistant:</b> {response}")  # Assistant's response
    )
    
    return True  # Continue the chatbot

# Function to handle input submission
def on_submit(change):
    condition = condition_input.value.strip()
    if condition:
        continue_chat = handle_chatbot_interaction(condition)
        if not continue_chat:
            condition_input.disabled = True  # Disable input after exit
    condition_input.value = ""  # Clear input field

# Widget for condition input
condition_input = widgets.Text(
    value='',
    placeholder='Enter a medical condition (or type help/exit)...',
    description='Condition:',
    layout=widgets.Layout(width='500px')
)

# Bind the function to the Text input submission event
condition_input.on_submit(on_submit)

# Initial welcome message
welcome_message = widgets.HTML("<b>Assistant:</b> Welcome! Type a medical condition or 'help' for assistance.")

# Display the input field and chatbox in Jupyter Notebook
display(welcome_message)
display(chatbox)
display(condition_input)


C:\Users\suru8\AppData\Local\Temp\ipykernel_4584\4190632467.py:121: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  condition_input.on_submit(on_submit)


HTML(value="<b>Assistant:</b> Welcome! Type a medical condition or 'help' for assistance.")

VBox()

Text(value='', description='Condition:', layout=Layout(width='500px'), placeholder='Enter a medical condition …

In [2]:
# Get unique conditions from the dataset
unique_conditions = combined_df['condition'].unique()

# Convert to a sorted list for readability
unique_conditions_list = sorted(unique_conditions)

# Display the unique conditions in a simple format
print("List of Conditions in the Dataset:")
for idx, condition in enumerate(unique_conditions_list, start=1):
    print(f"{idx}. {condition}")


List of Conditions in the Dataset:
1. 2 compressed discs in neck
2. a little bit of osteoporosis in the hips
3. ac joint dislocation
4. acid reflux
5. acid reflux / barrett's esophagus
6. acid reflux, gerd
7. acne
8. acne - blackheads and acne scars
9. acne / blackheads / oily skin
10. acne rosacea
11. acne with cystic fibrosis genetic condition
12. acne, aging
13. acne, anti-aging
14. acne, wrinkles
15. acne/dull skin
16. actinic keratoses
17. add
18. add and depression
19. addh
20. adhd
21. adhd (attention deficit hyperactive disorder)
22. adhd, inattentive type
23. adult acne
24. adult adhd
25. adult moderate hormone related acne
26. aging
27. aging skin
28. aging skin, few break-outs, hyperpigmentation
29. aging skin, wrinkles, sag
30. agitation management r/t lewy body syndrome  demen
31. agoraphobia, panic disorder
32. alcohol addiction
33. alergies
34. allegry
35. allergic rash
36. allergic reaction
37. allergic rhinitis
38. allergies
39. allergy
40. allergy, runny nose, hay fev